In [1]:
import tensorflow as tf
import pandas as pd
import numpy as np
import pickle

In [ ]:
## load the model
model = tf.keras.models.load_model('model/churn_model.h5')

## load the scaler
with open('scaler.pkl', 'rb') as f:
    scaler = pickle.load(f)

## load labelencoder
with open('le_gender.pkl', 'rb') as f:
    le_gender = pickle.load(f)
      
## load onehot
with open('ohe_geography.pkl', 'rb') as f:
    ohe_geography = pickle.load(f)

c:\Users\DELL\Desktop\deep_learning\venv\Lib\site-packages\keras\src\saving\saving_lib.py:797: UserWarning: Skipping variable loading for optimizer 'rmsprop', because it has 8 variables whereas the saved optimizer has 14 variables. 
  saveable.load_own_variables(weights_store.get(inner_path))


In [27]:
## example input data
input_data = {
    'CreditScore': 600,
    'Geography': 'France',
    'Gender': 'Male',
    'Age': 40,
    'Tenure': 3,
    'Balance': 60000,
    'NumOfProducts': 2,
    'HasCrCard': 1,
    'IsActiveMember': 1,
    'EstimatedSalary': 50000
}


In [30]:
## convert to dataframe
data = pd.DataFrame([input_data])


In [31]:

# Label Encode 'Gender'
gender_encoded = le_gender.transform(data[['Gender']].values.reshape(-1, 1))
data['Gender'] = gender_encoded.flatten()

# One-Hot Encoding for 'Geography'
geography_encoded = ohe_geography.transform(data[['Geography']])
df_geography_ohe = pd.DataFrame(geography_encoded, 
                                columns=ohe_geography.get_feature_names_out(['Geography']))

# Concatenate and drop the original 'Geography' column
data = pd.concat([data.drop('Geography', axis=1), df_geography_ohe], axis=1)

# Apply the fitted scaler. We use .values to get the NumPy array, excluding the index.
X_scaled = scaler.transform(data.values)

# --- 3. Final Prepared Input ---
print("\n--- Final Encoded and Scaled Data (ready for prediction) ---")
print("Shape:", X_scaled.shape)
print("Data:\n", X_scaled)


--- Final Encoded and Scaled Data (ready for prediction) ---
Shape: (1, 12)
Data:
 [[-0.53598516  0.91324755  0.10479359 -0.69539349 -0.25781119  0.80843615
   0.64920267  0.97481699 -0.87683221  1.00150113 -0.57946723 -0.57638802]]


c:\Users\DELL\Desktop\deep_learning\venv\Lib\site-packages\sklearn\preprocessing\_label.py:129: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, dtype=self.classes_.dtype, warn=True)
c:\Users\DELL\Desktop\deep_learning\venv\Lib\site-packages\sklearn\utils\validation.py:2749: UserWarning: X does not have valid feature names, but StandardScaler was fitted with feature names
  warnings.warn(


In [32]:
data

,CreditScore,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Geography_France,Geography_Germany,Geography_Spain
0,600,1,40,3,60000,2,1,1,50000,1.0,0.0,0.0


In [33]:
prediction = model.predict(X_scaled)
prediction

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 446ms/step


array([[0.03168725]], dtype=float32)

In [38]:
prediction_proba = prediction[0][0]
prediction_proba

np.float32(0.03168725)

In [39]:
if prediction_proba > 0.5:
    print('the customer is likely to churn')
else:
    print('the customer is not likey to churn')

the customer is not likey to churn


In [1]:
import tensorflow as tf
print(tf.__version__)


2.20.0
